### 문제 정의
- 백포도주의 열 가지 서로 다른 화학적 특성을 고려해 백포도주 데이터에 존재하는 알코올의 비율 예측 

#### Error 무시

In [11]:
import warnings

warnings.filterwarnings(action='ignore') 

### 라이브러리 import 

In [12]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf 

### 경로 설정 

In [15]:
TRAIN_DATA = './data/train/train_data.csv'
VAL_DATA = './data/val/val_data.csv'
TEST_DATA = './data/test/test_data.csv'

In [29]:
train = pd.read_csv(TRAIN_DATA)

train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,5.6,0.46,0.24,4.8,0.042,24.0,72.0,0.99080,3.29,0.37,12.6,6
1,6.5,0.18,0.41,14.2,0.039,47.0,129.0,0.99678,3.28,0.72,10.3,7
2,6.0,0.26,0.32,3.5,0.028,29.0,113.0,0.99120,3.40,0.71,12.3,7
3,7.4,0.18,0.40,1.6,0.047,22.0,102.0,0.99370,3.28,0.44,10.7,5
4,7.6,0.18,0.28,7.1,0.041,29.0,110.0,0.99652,3.20,0.42,9.2,6


In [28]:
train

[0, 1]

In [31]:
def load_data():
    train = pd.read_csv(TRAIN_DATA)
    val = pd.read_csv(VAL_DATA)
    test = pd.read_csv(TEST_DATA)
    
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    val = sclaer.fit_transform(val)
    test = scaler.fit_transform(test)
    
    data = dict()
    data['train_y'] = train[:, 10]
    data['train_x'] = train[:, 0:9]
    data['val_y'] = val[:, 10]
    data['val_x'] = val[:, 0:9]
    data['test_y'] = test[:, 10]
    data['test_x'] = test[:, 0:9] 
    
    # scaler를 유지함으로써 예측을 다시 원래 크기로 복원할 수 있도록 함 
    data['scaler'] = scaler 
    
    return data 